# 🚀 Brochure - Creation - Using LLMS

## 🌠Importing libraries

In [1]:
from openai import OpenAI
import os, json
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key or api_key[:8] != 'sk-proj-':
    print("Problem found with api key, please check the key in env file")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

## Extracting Info from 🕸️site using Class

In [3]:
class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Title Not Found"
        if soup.body:
            for irr in soup.body(["script", "style", "img", "input"]):
                irr.decompose()
            self.text = soup.body.get_text(separator = "\n", strip = True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:n {self.title} \n Webpage Contents: \n {self.text}\n\n"

In [4]:
# response = requests.get('https://edwarddonner.com/')
# soup = BeautifulSoup(body, 'html.parser')
# for ir in soup.body(["script", "style", "img", "input"]):
#     ir.decompose()
# soup.body

In [ ]:
url = 'https://cnn.com/'
obj = Website(url)
obj.links

## 📞 Creating API Call prompts for Links Segregation

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You have to decide which of the links are most relevant to include in a brochure for a company, \
such as links to About page, or company page. You should respond in JSON as this example :"
link_system_prompt += """
{
    "links": [
    {"type": "games page", "url" : "https://edition.cnn.com/games"},
    {"type": "profiles page", "url": "https://edition.cnn.com/profiles"}
}
"""

def get_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, repsond with the full https URL,\
    Do not include Terms of Service, Privacy, email links. \n"
    user_prompt += "Links (some mught be relevant links) : \n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
# print(get_links_user_prompt(obj))

## Getting Only the relevant Links 🔗

In [16]:
def get_links(url):
    website_obj = Website(url)
    response = openai.chat.completions.create(model = MODEL, messages = [
    {"role":"system", "content":link_system_prompt},
    {"role":"user", "content": get_user_prompt(website_obj)} ],
    response_format = {"type":"json_object"}
                                             )
    result = response.choices[0].message.content
    #print(result)
    return json.loads(result)

In [17]:
get_links(url)

{'links': [{'type': 'home page', 'url': 'https://edition.cnn.com'},
  {'type': 'about page', 'url': 'https://edition.cnn.com/about'},
  {'type': 'profiles page', 'url': 'https://edition.cnn.com/profiles'},
  {'type': 'leadership page',
   'url': 'https://edition.cnn.com/profiles/cnn-leadership'},
  {'type': 'careers page', 'url': 'https://careers.wbd.com/cnnjobs'},
  {'type': 'contact page', 'url': 'https://edition.cnn.com/newsletters'},
  {'type': 'social media', 'url': 'https://facebook.com/CNN'},
  {'type': 'social media', 'url': 'https://twitter.com/CNN'},
  {'type': 'social media', 'url': 'https://instagram.com/CNN'},
  {'type': 'social media', 'url': 'https://www.tiktok.com/@cnn?lang=en'},
  {'type': 'social media', 'url': 'https://www.linkedin.com/company/cnn'}]}

In [18]:
def get_details(url):
    result = "Page: \n"
    result += Website(url).get_contents()
    links = get_links(url)              ## Getting only the relevant links
    print("Found Links: ", links)
    for link in links["links"]:
        result += f"\n\n {link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_details("https://anthropic.com"))

In [19]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


def get_brochure_user_prompt(web_obj):
    prompt = f"You are looking at at a company called : {web_obj.title} \n"
    prompt += "Here are the details of it's page and other relevant pages: use this information to create a detailed but short brochure of the company\n"
    prompt += get_details(web_obj.url)
    prompt = prompt[:20000]  ## Truncate so that it does not exceed a limit, mostly to reduce costs
    return prompt
    

In [23]:
from IPython.display import Markdown, display, update_display
def create_brochure(url):
    response = openai.chat.completions.create(model = MODEL, messages = [{"role":"system", "content": system_prompt},
                                                                         {"role":"user", "content": get_brochure_user_prompt(Website(url))}])

    result = response.choices[0].message.content
    display(Markdown(result))

## Et Voila! 🪄

In [29]:
create_brochure('https://www.anthropic.com/')

# Anthropic: The Future of Safe AI

**About Us**
Anthropic is a pioneering AI safety and research company, dedicated to creating reliable, interpretable, and steerable AI systems. Our flagship product, Claude, embodies our commitment to ensuring that AI serves the long-term well-being of humanity. Through your partnership with us, you're joining a forward-thinking company that values safety, transparency, and ethical responsibility in AI development.

---

**Our Purpose**
We believe that AI will have a profound impact on our world. At Anthropic, we focus on building systems that not only advance AI capabilities but also prioritize human benefit. Our rigorous research and application of safety techniques ensure that the products we develop contribute positively to society.

---

**Our AI Models**
- **Claude Opus 4.1**: Our most powerful AI model yet, designed for high-level performance across various domains.
- **Claude Sonnet 4** and **Claude Haiku 3.5**: Diverse models optimizing different functionalities and applications.

---

**Safety as a Science**
At Anthropic, we treat AI safety as an empirical discipline. Our interdisciplinary team conducts research that informs our product development, aiming to set new standards for AI safety across industries.

---

**Company Culture**
We foster a collaborative environment where diverse perspectives thrive. Our team comprises researchers, engineers, policy experts, and operational leaders, all working towards the common goal of building safer AI systems. We value:

1. **Global Good**: Making decisions that maximize positive outcomes for humanity.
2. **Transparency**: Open communication about the impacts of our work.
3. **Utility and Kindness**: Being good to our users and the broader community.
4. **Innovation in Safety**: Promoting a culture where safety is prioritized in every project.

---

**Careers at Anthropic**
Join us in shaping the future of AI. We are always looking for passionate individuals in research, engineering, and policy-making to be part of our mission. Explore our open roles and be part of a high-impact team dedicated to making AI safer for everyone.

---

**Our Clients**
Our solutions cater to a diverse range of sectors including:
- **Customer Support**
- **Education**
- **Financial Services**
- **Government**

---

**Learn with Us**
Through the **Anthropic Academy**, we provide resources and training to help you and your team harness the power of Claude. Empower your organization with AI-driven efficiency and insights.

---

**Contact Us**
Interested in collaborating or learning more? Reach out to us today and discover how Anthropic can help you harness the future of safe AI. 

**Let’s build a better tomorrow, together!**

--- 

For more information, visit our website at [Anthropic](https://www.anthropic.com) or explore our product offerings via Claude.ai.

In [32]:
def stream_brochure(url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(Website(url))}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        #response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure('https://www.anthropic.com/')

Found Links:  {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'api page', 'url': 'https://www.anthropic.com/api'}]}


# Anthropic: Building Safe and Reliable AI

---

## About Us
Anthropic is at the forefront of AI safety and research, dedicated to creating reliable, interpretable, and steerable AI systems that prioritize human well-being. Our cornerstone product, **Claude**, embodies our mission to build AI technologies that serve the greater good, leveraging our extensive research to create tools that empower various sectors including education, finance, government, and customer support.

### Our Philosophy
We believe that AI holds the potential to significantly impact society—positively and negatively. That's why we focus on designing AI technologies that are safe and beneficial for everyone. Our multidisciplinary approach combines insights from engineering, policy, and research, fostering collaboration to promote the safe development of AI worldwide.

---

## Our Products
### Meet Claude
Claude, the AI assistant designed for seamless collaboration, enhances productivity by utilizing your team's combined expertise. Key features include:
- **Collaborative Assistance**: Helps with writing, coding, and daily tasks like drafting emails.
- **Project Management**: Ground Claude in specific knowledge for enhanced outcomes.
- **Creativity Booster**: Generates ideas and insights quickly to spark innovation across your teams.

### Product Offerings
- **Claude Opus 4.1**: Our most advanced AI model designed for complex tasks across various domains.
- **Claude Code**: Tailored for engineers to improve coding processes and collaboration.
- **Enterprise Solutions**: Custom plans to suit large organizations, enabling teamwork and maximizing productivity.

---

## Customer Commitment
Our mission extends beyond merely providing AI solutions; we strive to build lasting relationships with our diverse clientele that ranges from small start-ups to large enterprises. Through our commitment to transparency and security, we engage in ongoing dialogues with our users to understand their needs and enhance their experience.

**Our Customers Include:**
- Educational institutions
- Financial services corporations
- Government agencies
- Non-profit organizations

---

## Join the Anthropic Team
At Anthropic, we cherish a collaborative and inclusive company culture that emphasizes personal growth and a sense of responsibility towards societal well-being. Our values guide our daily decisions and actions:

1. **Act for the global good**.
2. **Hold light and shade**: Recognize both the risks and benefits of AI.
3. **Be good to our users**: Cultivate kindness and generosity.
4. **Ignite a race to the top on safety**: Set industry standards for AI safety.
5. **Do the simple thing that works**.
6. **Be helpful, honest, and harmless**.
7. **Put the mission first**.

### Careers At Anthropic
Join a team of passionate experts in AI research, engineering, policy-making, and operations. We are looking for individuals who are eager to contribute to the safe development of AI technologies. Explore our open roles on our website and help shape the future of AI.

---

## Connect With Us
Together, let's navigate the promising yet complex landscape of AI. For more information, visit our website at [Anthropic](https://www.anthropic.com) or reach out to us directly for inquiries.

---

Join us in our mission to develop AI that prioritizes safety and enriches human life. **Together, we can build a safer future with AI.**